# Generating Team Data
In this notebook, file we will be generating statistics on a team basis. This will be better for training a model as it will contain avergaes for teams **going into** a match up.

# Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from sportsreference.ncaab.teams import Teams

pd.set_option("display.max_rows", None, "display.max_columns", None)

# Utils

In [2]:
def clean_dir(path):
    """Makes a clean directory, removes all files and folders in the specified path"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

# Generating Moving Averages

Next, we want to create a file for each team that contains all of their stats for each game.

In [3]:
TEAMS = ['Abilene Christian', 'Air Force', 'Akron', 'Alabama', 'Alabama A&M', 'Alabama State', 'Alabama-Birmingham', 'Albany (NY)', 'Alcorn State', 'American', 'Appalachian State', 'Arizona', 'Arizona State', 'Arkansas', 'Arkansas State', 'Arkansas-Pine Bluff', 'Army', 'Auburn', 'Austin Peay', 'Ball State', 'Baylor', 'Bellarmine', 'Belmont', 'Binghamton', 'Boise State', 'Boston College', 'Boston University', 'Bowling Green State', 'Bradley', 'Brigham Young', 'Brown', 'Bryant', 'Bucknell', 'Buffalo', 'Butler', 'Cal Poly', 'Cal State Bakersfield', 'Cal State Fullerton', 'Cal State Long Beach', 'Cal State Northridge', 'California Baptist', 'Campbell', 'Canisius', 'Central Arkansas', 'Central Connecticut State', 'Central Florida', 'Central Michigan', 'Charleston Southern', 'Charlotte', 'Chattanooga', 'Chicago State', 'Cincinnati', 'Citadel', 'Clemson', 'Cleveland State', 'Coastal Carolina', 'Colgate', 'College of Charleston', 'Colorado', 'Colorado State', 'Columbia', 'Connecticut', 'Coppin State', 'Cornell', 'Creighton', 'Dartmouth', 'Davidson', 'Dayton', 'DePaul', 'Delaware', 'Delaware State', 'Denver', 'Detroit Mercy', 'Dixie State', 'Drake', 'Drexel', 'Duke', 'Duquesne', 'East Carolina', 'East Tennessee State', 'Eastern Illinois', 'Eastern Kentucky', 'Eastern Michigan', 'Eastern Washington', 'Elon', 'Evansville', 'Fairfield', 'Fairleigh Dickinson', 'Florida', 'Florida A&M', 'Florida Atlantic', 'Florida Gulf Coast', 'Florida International', 'Florida State', 'Fordham', 'Fresno State', 'Furman', 'Gardner-Webb', 'George Mason', 'George Washington', 'Georgetown', 'Georgia', 'Georgia Southern', 'Georgia State', 'Georgia Tech', 'Gonzaga', 'Grambling', 'Grand Canyon', 'Green Bay', 'Hampton', 'Hartford', 'Harvard', 'Hawaii', 'High Point', 'Hofstra', 'Holy Cross', 'Houston', 'Houston Baptist', 'Howard', 'IUPUI', 'Idaho', 'Idaho State', 'Illinois', 'Illinois State', 'Illinois-Chicago', 'Incarnate Word', 'Indiana', 'Indiana State', 'Iona', 'Iowa', 'Iowa State', 'Jackson State', 'Jacksonville', 'Jacksonville State', 'James Madison', 'Kansas', 'Kansas State', 'Kennesaw State', 'Kent State', 'Kentucky', 'La Salle', 'Lafayette', 'Lamar', 'Lehigh', 'Liberty', 'Lipscomb', 'Little Rock', 'Long Island University', 'Longwood', 'Louisiana', 'Louisiana State', 'Louisiana Tech', 'Louisiana-Monroe', 'Louisville', 'Loyola (IL)', 'Loyola (MD)', 'Loyola Marymount', 'Maine', 'Manhattan', 'Marist', 'Marquette', 'Marshall', 'Maryland', 'Maryland-Baltimore County', 'Maryland-Eastern Shore', 'Massachusetts', 'Massachusetts-Lowell', 'McNeese State', 'Memphis', 'Mercer', 'Merrimack', 'Miami (FL)', 'Miami (OH)', 'Michigan', 'Michigan State', 'Middle Tennessee', 'Milwaukee', 'Minnesota', 'Mississippi', 'Mississippi State', 'Mississippi Valley State', 'Missouri', 'Missouri State', 'Missouri-Kansas City', 'Monmouth', 'Montana', 'Montana State', 'Morehead State', 'Morgan State', "Mount St. Mary's", 'Murray State', 'NJIT', 'Navy', 'Nebraska', 'Nevada', 'Nevada-Las Vegas', 'New Hampshire', 'New Mexico', 'New Mexico State', 'New Orleans', 'Niagara', 'Nicholls State', 'Norfolk State', 'North Alabama', 'North Carolina', 'North Carolina A&T', 'North Carolina Central', 'North Carolina State', 'North Carolina-Asheville', 'North Carolina-Greensboro', 'North Carolina-Wilmington', 'North Dakota', 'North Dakota State', 'North Florida', 'North Texas', 'Northeastern', 'Northern Arizona', 'Northern Colorado', 'Northern Illinois', 'Northern Iowa', 'Northern Kentucky', 'Northwestern', 'Northwestern State', 'Notre Dame', 'Oakland', 'Ohio', 'Ohio State', 'Oklahoma', 'Oklahoma State', 'Old Dominion', 'Omaha', 'Oral Roberts', 'Oregon', 'Oregon State', 'Pacific', 'Penn State', 'Pennsylvania', 'Pepperdine', 'Pittsburgh', 'Portland', 'Portland State', 'Prairie View', 'Presbyterian', 'Princeton', 'Providence', 'Purdue', 'Purdue-Fort Wayne', 'Quinnipiac', 'Radford', 'Rhode Island', 'Rice', 'Richmond', 'Rider', 'Robert Morris', 'Rutgers', 'SIU Edwardsville', 'Sacramento State', 'Sacred Heart', 'Saint Francis (PA)', "Saint Joseph's", 'Saint Louis', "Saint Mary's (CA)", "Saint Peter's", 'Sam Houston State', 'Samford', 'San Diego', 'San Diego State', 'San Francisco', 'San Jose State', 'Santa Clara', 'Seattle', 'Seton Hall', 'Siena', 'South Alabama', 'South Carolina', 'South Carolina State', 'South Carolina Upstate', 'South Dakota', 'South Dakota State', 'South Florida', 'Southeast Missouri State', 'Southeastern Louisiana', 'Southern', 'Southern California', 'Southern Illinois', 'Southern Methodist', 'Southern Mississippi', 'Southern Utah', 'St. Bonaventure', 'St. Francis (NY)', "St. John's (NY)", 'Stanford', 'Stephen F. Austin', 'Stetson', 'Stony Brook', 'Syracuse', 'Tarleton State', 'Temple', 'Tennessee', 'Tennessee State', 'Tennessee Tech', 'Tennessee-Martin', 'Texas', 'Texas A&M', 'Texas A&M-Corpus Christi', 'Texas Christian', 'Texas Southern', 'Texas State', 'Texas Tech', 'Texas-Arlington', 'Texas-El Paso', 'Texas-Rio Grande Valley', 'Texas-San Antonio', 'Toledo', 'Towson', 'Troy', 'Tulane', 'Tulsa', 'UC-Davis', 'UC-Irvine', 'UC-Riverside', 'UC-San Diego', 'UC-Santa Barbara', 'UCLA', 'University of California', 'Utah', 'Utah State', 'Utah Valley', 'VMI', 'Valparaiso', 'Vanderbilt', 'Vermont', 'Villanova', 'Virginia', 'Virginia Commonwealth', 'Virginia Tech', 'Wagner', 'Wake Forest', 'Washington', 'Washington State', 'Weber State', 'West Virginia', 'Western Carolina', 'Western Illinois', 'Western Kentucky', 'Western Michigan', 'Wichita State', 'William & Mary', 'Winthrop', 'Wisconsin', 'Wofford', 'Wright State', 'Wyoming', 'Xavier', 'Yale', 'Youngstown State']

In [4]:
original_data = pd.read_csv('../assets/data/data.csv')
original_data["away"] = np.where(original_data["winner"]=="Away",
                                 original_data["winning_name"],
                                 original_data["losing_name"])

original_data["home"] = np.where(original_data["winner"]=="Home",
                                 original_data["winning_name"],
                                 original_data["losing_name"])

# This will help later
reordered_labels = ['date',
                    'location',
                    'losing_abbr',
                    'losing_name',
                    'pace',
                    'winning_abbr',
                    'winning_name',
                    'away',
                    'home',
                    'winner',
                    'away_assist_percentage',
                    'away_assists', 
                    'away_block_percentage', 
                    'away_blocks', 
                    'away_defensive_rating', 
                    'away_defensive_rebound_percentage', 
                    'away_defensive_rebounds',
                    'away_effective_field_goal_percentage',
                    'away_field_goal_attempts',
                    'away_field_goal_percentage',
                    'away_field_goals',
                    'away_free_throw_attempt_rate',
                    'away_free_throw_attempts',
                    'away_free_throw_percentage',
                    'away_free_throws',
                    'away_losses',
                    'away_minutes_played',
                    'away_offensive_rating',
                    'away_offensive_rebound_percentage',
                    'away_offensive_rebounds',
                    'away_personal_fouls',
                    'away_points',
                    'away_steal_percentage',
                    'away_steals',
                    'away_three_point_attempt_rate',
                    'away_three_point_field_goal_attempts',
                    'away_three_point_field_goal_percentage',
                    'away_three_point_field_goals',
                    'away_total_rebound_percentage',
                    'away_total_rebounds',
                    'away_true_shooting_percentage',
                    'away_turnover_percentage',
                    'away_turnovers',
                    'away_two_point_field_goal_attempts',
                    'away_two_point_field_goal_percentage',
                    'away_two_point_field_goals',
                    'away_win_percentage',
                    'away_wins',
                    'home_assist_percentage',
                    'home_assists',
                    'home_block_percentage',
                    'home_blocks',
                    'home_defensive_rating',
                    'home_defensive_rebound_percentage',
                    'home_defensive_rebounds',
                    'home_effective_field_goal_percentage',
                    'home_field_goal_attempts',
                    'home_field_goal_percentage',
                    'home_field_goals',
                    'home_free_throw_attempt_rate',
                    'home_free_throw_attempts',
                    'home_free_throw_percentage',
                    'home_free_throws',
                    'home_losses',
                    'home_minutes_played',
                    'home_offensive_rating',
                    'home_offensive_rebound_percentage',
                    'home_offensive_rebounds',
                    'home_personal_fouls',
                    'home_points',
                    'home_steal_percentage',
                    'home_steals',
                    'home_three_point_attempt_rate',
                    'home_three_point_field_goal_attempts',
                    'home_three_point_field_goal_percentage',
                    'home_three_point_field_goals',
                    'home_total_rebound_percentage',
                    'home_total_rebounds',
                    'home_true_shooting_percentage',
                    'home_turnover_percentage',
                    'home_turnovers',
                    'home_two_point_field_goal_attempts',
                    'home_two_point_field_goal_percentage',
                    'home_two_point_field_goals',
                    'home_win_percentage',
                    'home_wins']

original_data.drop(columns=['away_ranking', 'home_ranking'], inplace=True)
original_data['date'] = pd.to_datetime(original_data['date'])
original_data['date'] = original_data['date'].dt.strftime('%Y-%m-%d')
original_data = original_data[reordered_labels]
original_data.dropna(inplace=True)
original_data.drop_duplicates(inplace=True)

print(f"There are {original_data.shape[0]} examples in the set")

There are 5747 examples in the set


In [5]:
new_labels = ['date',
              'location',
              'away',
              'home',
              'winner',
              'assist_percentage',
              'assists', 
              'block_percentage', 
              'blocks', 
              'defensive_rating', 
              'defensive_rebound_percentage', 
              'defensive_rebounds',
              'effective_field_goal_percentage',
              'field_goal_attempts',
              'field_goal_percentage',
              'field_goals',
              'free_throw_attempt_rate',
              'free_throw_attempts',
              'free_throw_percentage',
              'free_throws',
              'losses',
              'minutes_played',
              'offensive_rating',
              'offensive_rebound_percentage',
              'offensive_rebounds',
              'personal_fouls',
              'points',
              'steal_percentage',
              'steals',
              'three_point_attempt_rate',
              'three_point_field_goal_attempts',
              'three_point_field_goal_percentage',
              'three_point_field_goals',
              'total_rebound_percentage',
              'total_rebounds',
              'true_shooting_percentage',
              'turnover_percentage',
              'turnovers',
              'two_point_field_goal_attempts',
              'two_point_field_goal_percentage',
              'two_point_field_goals',
              'win_percentage',
              'wins']

def generateTeamSats(dataframe, teams=TEAMS, folder='../assets/data/team_data/'):
    """For each team, generate the games that they have played"""
    
    # Clean the folder
    clean_dir(folder)
    
    # Loop through all of the teams
    for team in tqdm(teams, unit="teams"):
        file_name = f'{team}_data.csv'
        path = os.path.join(folder, file_name)
        
        # Get home games and away games
        away_games = dataframe.loc[(dataframe["away"] == team)].copy()
        home_games = dataframe.loc[(dataframe["home"] == team)].copy()
        
        home_games.drop(home_games.filter(regex="away_").columns, axis=1, inplace=True)
        home_games.drop(columns=['losing_abbr',
                                 'losing_name',
                                 'pace',
                                 'winning_abbr',
                                 'winning_name'],
                        inplace=True)
        home_games.columns = new_labels
        
        away_games.drop(away_games.filter(regex="home_").columns, axis=1, inplace=True)
        away_games.drop(columns=['losing_abbr',
                                 'losing_name',
                                 'pace',
                                 'winning_abbr',
                                 'winning_name'],
                        inplace=True)
        away_games.columns = new_labels
        
        # Shaoe for both must match same columns
        assert away_games.shape[1] == home_games.shape[1]
        
        # Join the home games and away games, sort by date
        team_stats = pd.concat([home_games, away_games])
        team_stats.drop_duplicates(inplace=True)
        team_stats.dropna(inplace=True)
        team_stats.sort_values(by=["date"], inplace=True)
        
        # Save the stats
        team_stats.to_csv(path, index=None)

In [6]:
generateTeamSats(original_data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 356/356 [00:07<00:00, 47.66teams/s]


In [7]:
pd.read_csv('../assets/data/team_data/Yale_data.csv').head()

,date,location,away,home,winner,assist_percentage,assists,block_percentage,blocks,defensive_rating,defensive_rebound_percentage,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,losses,minutes_played,offensive_rating,offensive_rebound_percentage,offensive_rebounds,personal_fouls,points,steal_percentage,steals,three_point_attempt_rate,three_point_field_goal_attempts,three_point_field_goal_percentage,three_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,win_percentage,wins
0,2019-11-05,"Island Federal Credit Union, Stony Brook, New ...",Yale,Stony Brook,Away,59.3,16,15.6,7,84.1,71.1,32,0.450,70,0.386,27,0.271,19,0.579,11,0,225,90.2,25.0,11,18,74,9.8,8,0.371,26,0.346,9,48.3,43,0.468,13.3,12,44,0.409,18,1.000,1
1,2019-11-08,"John J. Lee Ampitheater, New Haven, Connecticut",Oberlin\r\n\t\t\t,Yale,Home,73.3,22,19.2,5,50.7,87.9,29,0.632,53,0.566,30,0.811,43,0.628,27,0,200,128.8,50.0,13,12,94,15.1,11,0.302,16,0.438,7,71.2,42,0.640,17.3,15,37,0.622,23,1.000,2
2,2019-11-11,"War Memorial Gymnasium, San Francisco, California",Yale,San Francisco,Home,41.7,10,9.8,4,106.3,71.8,28,0.460,63,0.381,24,0.444,28,0.750,21,1,225,100.0,12.8,5,20,79,6.3,5,0.429,27,0.370,10,42.3,33,0.518,8.5,7,36,0.389,14,0.667,2
3,2019-11-17,"Gallagher-Iba Arena, Stillwater, Oklahoma",Yale,Oklahoma State,Home,61.9,13,8.7,4,88.9,75.0,30,0.464,55,0.382,21,0.164,9,0.667,6,2,200,79.2,26.5,9,16,57,6.9,5,0.564,31,0.290,9,52.7,39,0.481,25.3,20,24,0.500,12,0.500,2
4,2019-11-20,"John J. Lee Ampitheater, New Haven, Connecticut",Siena,Yale,Home,74.3,26,16.7,10,95.7,72.2,39,0.603,68,0.515,35,0.412,28,0.643,18,2,275,107.5,12.8,5,23,100,3.2,3,0.426,29,0.414,12,47.3,44,0.615,16.6,16,39,0.590,23,0.600,3


In [8]:
def compute_ma(span, teams=TEAMS, team_data_folder='../assets/data/team_data/', dest_folder='../assets/data/team_ma/'):
    """Computes various moving averages for the stats"""
    
    # Clean directory we will be saving the CSVs to
    clean_dir(dest_folder)
    
    for team in tqdm(teams, unit='teams'):
        averages = pd.DataFrame()

        # Load the stats for a given team
        team_stats = pd.read_csv(os.path.join(team_data_folder, f"{team}_data.csv"))

        # Compute the moving averages for the appropriate columns
        for col in team_stats.columns:
            if col in {'date', 'location', 'away', 'home', 'winner'}:
                continue
                
            # Simple moving average
            team_stats[f"{col}_SMA"] = team_stats.loc[:, col].rolling(window=span).mean()
            team_stats[f"{col}_SMA"] = team_stats[f"{col}_SMA"].shift(1)

            # Cumulative moving average
            team_stats[f"{col}_CMA"] = team_stats.loc[:, col].expanding(min_periods=span).mean()
            team_stats[f"{col}_CMA"] = team_stats[f"{col}_CMA"].shift(1)

            # Exponential moving average
            team_stats[f"{col}_EMA"] = team_stats.loc[:, col].ewm(span=span, adjust=False).mean()
            team_stats[f"{col}_EMA"] = team_stats[f"{col}_EMA"].shift(1)

        # Drop any rows with NULL values and save the CSV
        team_stats.dropna(inplace=True)
        team_stats.drop_duplicates(inplace=True)
        team_stats.to_csv(os.path.join(dest_folder, f"{team}_ma.csv"), index=None)

In [9]:
compute_ma(5)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 356/356 [00:37<00:00,  9.43teams/s]


In [10]:
pd.read_csv('../assets/data/team_ma/Yale_ma.csv').head()

,date,location,away,home,winner,assist_percentage,assists,block_percentage,blocks,defensive_rating,defensive_rebound_percentage,defensive_rebounds,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,losses,minutes_played,offensive_rating,offensive_rebound_percentage,offensive_rebounds,personal_fouls,points,steal_percentage,steals,three_point_attempt_rate,three_point_field_goal_attempts,three_point_field_goal_percentage,three_point_field_goals,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,win_percentage,wins,assist_percentage_SMA,assist_percentage_CMA,assist_percentage_EMA,assists_SMA,assists_CMA,assists_EMA,block_percentage_SMA,block_percentage_CMA,block_percentage_EMA,blocks_SMA,blocks_CMA,blocks_EMA,defensive_rating_SMA,defensive_rating_CMA,defensive_rating_EMA,defensive_rebound_percentage_SMA,defensive_rebound_percentage_CMA,defensive_rebound_percentage_EMA,defensive_rebounds_SMA,defensive_rebounds_CMA,defensive_rebounds_EMA,effective_field_goal_percentage_SMA,effective_field_goal_percentage_CMA,effective_field_goal_percentage_EMA,field_goal_attempts_SMA,field_goal_attempts_CMA,field_goal_attempts_EMA,field_goal_percentage_SMA,field_goal_percentage_CMA,field_goal_percentage_EMA,field_goals_SMA,field_goals_CMA,field_goals_EMA,free_throw_attempt_rate_SMA,free_throw_attempt_rate_CMA,free_throw_attempt_rate_EMA,free_throw_attempts_SMA,free_throw_attempts_CMA,free_throw_attempts_EMA,free_throw_percentage_SMA,free_throw_percentage_CMA,free_throw_percentage_EMA,free_throws_SMA,free_throws_CMA,free_throws_EMA,losses_SMA,losses_CMA,losses_EMA,minutes_played_SMA,minutes_played_CMA,minutes_played_EMA,offensive_rating_SMA,offensive_rating_CMA,offensive_rating_EMA,offensive_rebound_percentage_SMA,offensive_rebound_percentage_CMA,offensive_rebound_percentage_EMA,offensive_rebounds_SMA,offensive_rebounds_CMA,offensive_rebounds_EMA,personal_fouls_SMA,personal_fouls_CMA,personal_fouls_EMA,points_SMA,points_CMA,points_EMA,steal_percentage_SMA,steal_percentage_CMA,steal_percentage_EMA,steals_SMA,steals_CMA,steals_EMA,three_point_attempt_rate_SMA,three_point_attempt_rate_CMA,three_point_attempt_rate_EMA,three_point_field_goal_attempts_SMA,three_point_field_goal_attempts_CMA,three_point_field_goal_attempts_EMA,three_point_field_goal_percentage_SMA,three_point_field_goal_percentage_CMA,three_point_field_goal_percentage_EMA,three_point_field_goals_SMA,three_point_field_goals_CMA,three_point_field_goals_EMA,total_rebound_percentage_SMA,total_rebound_percentage_CMA,total_rebound_percentage_EMA,total_rebounds_SMA,total_rebounds_CMA,total_rebounds_EMA,true_shooting_percentage_SMA,true_shooting_percentage_CMA,true_shooting_percentage_EMA,turnover_percentage_SMA,turnover_percentage_CMA,turnover_percentage_EMA,turnovers_SMA,turnovers_CMA,turnovers_EMA,two_point_field_goal_attempts_SMA,two_point_field_goal_attempts_CMA,two_point_field_goal_attempts_EMA,two_point_field_goal_percentage_SMA,two_point_field_goal_percentage_CMA,two_point_field_goal_percentage_EMA,two_point_field_goals_SMA,two_point_field_goals_CMA,two_point_field_goals_EMA,win_percentage_SMA,win_percentage_CMA,win_percentage_EMA,wins_SMA,wins_CMA,wins_EMA
0,2019-11-23,"Bryce Jordan Center, University Park, Pennsylv...",Yale,Penn State,Home,45.5,10,9.5,4,86.6,68.9,31,0.520,51,0.431,22,0.216,11,0.273,3,3,200,83.6,9.4,3,15,56,3.0,2,0.451,23,0.391,9,44.2,34,0.498,21.2,15,28,0.464,13,0.500,3,62.10,62.100000,63.653086,17.4,17.400000,18.370370,14.00,14.000000,13.929630,6.0,6.000000,6.691358,85.14,85.140000,89.023457,75.60,75.600000,74.096296,31.6,31.600000,33.000000,0.5218,0.521800,0.523568,61.8,61.800000,63.283951,0.4460,0.446000,0.445148,27.4,27.400000,28.185185,0.4204,0.420400,0.373185,25.4,25.400000,23.481481,0.6534,0.653400,0.650062,16.6,16.600000,15.283

# Getting the Data Ready for Testing
Now that we have computed the exponential moving average for each team, we need to joing them back into one table so that it is easier to train models on.

In [11]:
games = original_data.copy()
num_rows = games.shape[0]

# We can uniquely identify a game given the teams, the date, and the game number.
games = games.filter(["date", "location", "home", "away", "winner"])
games.drop_duplicates()

# Just to make sure that there is no lost data
assert games.shape[0] == num_rows

# Arrays to hold the home and away dataframes
homes=[]
aways=[]

# Add the moving averages
for team in tqdm(TEAMS, unit='teams'):
    # load in the  file
    team_avgs = pd.read_csv(f'../assets/data/team_ma/{team}_ma.csv')
    team_avgs.drop(new_labels[5:], axis=1, inplace=True)
    
    # Merge in visiting team stats
    away = pd.merge(games,
                    team_avgs.loc[team_avgs["away"] == team],
                    on=["date", "location", "home", "away", 'winner'],
                    how="left")
    
    # Drop any rows with NULL values and drop redudant columns
    away.dropna(inplace=True)
    away.drop_duplicates(inplace=True)
    
    # Rename the columns and append to the array
    new_away_cols = list(away.columns)[:5] + ["away_" + col for col in list(away.columns)[5:]]
    away.columns = new_away_cols
    aways.append(away)
    
    # Merge in home team stats
    home = pd.merge(games,
                    team_avgs.loc[team_avgs["home"] == team],
                    on=["date", "location", "home", "away", 'winner'],
                    how="left")
    
    # Drop any rows with NULL values and drop redudant columns
    home.dropna(inplace=True)
    home.drop_duplicates(inplace=True)
    
    # Rename the columns and append to the array
    new_home_cols = list(home.columns)[:5] + ["home_" + col for col in list(home.columns)[5:]]
    home.columns = new_home_cols
    homes.append(home)
    
    # Sanity check, make sure that the column sizes match for both home and away dataframes
    assert home.shape[1] == away.shape[1]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 356/356 [00:27<00:00, 12.88teams/s]


In [12]:
# Join the tables on the common attributes and sort by the Date and the Number of game
data = pd.merge(pd.concat(homes),
                pd.concat(aways),
                on=["date", "location", "home", "away", "winner"]).sort_values(["date"]).reset_index(drop=True)

data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.head(10)

,date,location,home,away,winner,home_assist_percentage_SMA,home_assist_percentage_CMA,home_assist_percentage_EMA,home_assists_SMA,home_assists_CMA,home_assists_EMA,home_block_percentage_SMA,home_block_percentage_CMA,home_block_percentage_EMA,home_blocks_SMA,home_blocks_CMA,home_blocks_EMA,home_defensive_rating_SMA,home_defensive_rating_CMA,home_defensive_rating_EMA,home_defensive_rebound_percentage_SMA,home_defensive_rebound_percentage_CMA,home_defensive_rebound_percentage_EMA,home_defensive_rebounds_SMA,home_defensive_rebounds_CMA,home_defensive_rebounds_EMA,home_effective_field_goal_percentage_SMA,home_effective_field_goal_percentage_CMA,home_effective_field_goal_percentage_EMA,home_field_goal_attempts_SMA,home_field_goal_attempts_CMA,home_field_goal_attempts_EMA,home_field_goal_percentage_SMA,home_field_goal_percentage_CMA,home_field_goal_percentage_EMA,home_field_goals_SMA,home_field_goals_CMA,home_field_goals_EMA,home_free_throw_attempt_rate_SMA,home_free_throw_attempt_rate_CMA,home_free_throw_attempt_rate_EMA,home_free_throw_attempts_SMA,home_free_throw_attempts_CMA,home_free_throw_attempts_EMA,home_free_throw_percentage_SMA,home_free_throw_percentage_CMA,home_free_throw_percentage_EMA,home_free_throws_SMA,home_free_throws_CMA,home_free_throws_EMA,home_losses_SMA,home_losses_CMA,home_losses_EMA,home_minutes_played_SMA,home_minutes_played_CMA,home_minutes_played_EMA,home_offensive_rating_SMA,home_offensive_rating_CMA,home_offensive_rating_EMA,home_offensive_rebound_percentage_SMA,home_offensive_rebound_percentage_CMA,home_offensive_rebound_percentage_EMA,home_offensive_rebounds_SMA,home_offensive_rebounds_CMA,home_offensive_rebounds_EMA,home_personal_fouls_SMA,home_personal_fouls_CMA,home_personal_fouls_EMA,home_points_SMA,home_points_CMA,home_points_EMA,home_steal_percentage_SMA,home_steal_percentage_CMA,home_steal_percentage_EMA,home_steals_SMA,home_steals_CMA,home_steals_EMA,home_three_point_attempt_rate_SMA,home_three_point_attempt_rate_CMA,home_three_point_attempt_rate_EMA,home_three_point_field_goal_attempts_SMA,home_three_point_field_goal_attempts_CMA,home_three_point_field_goal_attempts_EMA,home_three_point_field_goal_percentage_SMA,home_three_point_field_goal_percentage_CMA,home_three_point_field_goal_percentage_EMA,home_three_point_field_goals_SMA,home_three_point_field_goals_CMA,home_three_point_field_goals_EMA,home_total_rebound_percentage_SMA,home_total_rebound_percentage_CMA,home_total_rebound_percentage_EMA,home_total_rebounds_SMA,home_total_rebounds_CMA,home_total_rebounds_EMA,home_true_shooting_percentage_SMA,home_true_shooting_percentage_CMA,home_true_shooting_percentage_EMA,home_turnover_percentage_SMA,home_turnover_percentage_CMA,home_turnover_percentage_EMA,home_turnovers_SMA,home_turnovers_CMA,home_turnovers_EMA,home_two_point_field_goal_attempts_SMA,home_two_point_field_goal_attempts_CMA,home_two_point_field_goal_attempts_EMA,home_two_point_field_goal_percentage_SMA,home_two_point_field_goal_percentage_CMA,home_two_point_field_goal_percentage_EMA,home_two_point_field_goals_SMA,home_two_point_field_goals_CMA,home_two_point_field_goals_EMA,home_win_percentage_SMA,home_win_percentage_CMA,home_win_percentage_EMA,home_wins_SMA,home_wins_CMA,home_wins_EMA,away_assist_percentage_SMA,away_assist_percentage_CMA,away_assist_percentage_EMA,away_assists_SMA,away_assists_CMA,away_assists_EMA,away_block_percentage_SMA,away_block_percentage_CMA,away_block_percentage_EMA,away_blocks_SMA,away_blocks_CMA,away_blocks_EMA,away_defensive_rating_SMA,away_defensive_rating_CMA,away_defensive_rating_EMA,away_defensive_rebound_percentage_SMA,away_defensive_rebound_percentage_CMA,away_defensive_rebound_percentage_EMA,away_defensive_rebounds_SMA,away_defensive_rebounds_CMA,away_defensive_rebounds_EMA,away_effective_field_goal_percentage_SMA,away_effective_field_goal_percentage_CMA,away_effective_field_goal_percentage_EMA,away_field_goal_attempts_SMA,away_field_goal_attempts_CMA,away_field_goal_attempts_EMA,away_field_goal_perc

In [13]:
# Save the dataframe that we just generated

clean_dir('../assets/data/cleaned_data')
data.to_csv('../assets/data/cleaned_data/cleaned_data.csv', index=None)